# Get Forever 21 locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "forever-21"
place_formal = "Forever 21"
color = "#837700"

## Scrape

#### Headers for request

In [4]:
headers = {
    "authority": "locations.forever21.com",
    "accept": "application/json",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

#### The locatin API only returns 10 locations per request. Determine our pagination.

In [5]:
pages = requests.get(
    "https://locations.forever21.com/index.html",
    # params=params,
    headers=headers,
)

results_count = int(pages.json()["response"]["count"])

In [6]:
results_list = []

for page in tqdm(range(0, results_count, 10)):
    params = {
        "offset": page,
    }

    response = requests.get(
        "https://locations.forever21.com/index.html",
        params=params,
        headers=headers,
    ).json()["response"]["entities"]

    for r in response:
        address = r["profile"]["address"]["line1"]
        city = r["profile"]["address"]["city"]
        state = r["profile"]["address"]["region"]
        zip = r["profile"]["address"]["postalCode"]
        phone = r["profile"]["mainPhone"]["display"]
        try:
            url = r["profile"]["websiteUrl"]
        except:
            continue
        latitude = r["profile"]["yextDisplayCoordinate"]["lat"]
        longitude = r["profile"]["yextDisplayCoordinate"]["long"]

        results_dict = {
            "address": address,
            "city": city,
            "state": state,
            "zip": zip,
            "phone": phone,
            "url": url.replace(
                "?utm_medium=organic&utm_source=local&utm_campaign=other", ""
            ),
            "latitude": latitude,
            "longitude": longitude,
        }

        results_list.append(results_dict)

  0%|          | 0/40 [00:00<?, ?it/s]

#### Read list of dictionaries into a dataframe

In [7]:
df = pd.DataFrame(results_list).drop_duplicates()

#### The result

In [8]:
df.head()

,address,city,state,zip,phone,url,latitude,longitude
0,200 Inland Dr,San Bernardino,CA,92408,(909) 915-2996,https://locations.forever21.com/us/stores/ca/sanbernardino/200-inland-dr,34.085060,-117.297286
1,200 Los Cerritos Mall,Cerritos,CA,90703,(562) 356-6959,https://locations.forever21.com/us/stores/ca/cerritos/200-los-cerritos-mall,33.862166,-118.095430
2,326 Lakewood Center Mall,Lakewood,CA,90712,(562) 280-7435,https://locations.forever21.com/us/stores/ca/lakewood/326-lakewood-center-mall,33.851036,-118.138589
3,1201 Lake Woodlands Dr,Spring,TX,77380,(346) 224-9779,https://locations.forever21.com/us/stores/tx/spring/1201-lake-woodlands-dr,30.164003,-95.456675
4,3200 Las Vegas Blvd,Las Vegas,NV,89109,(702) 680-6372,https://locations.forever21.com/us/stores/nv/lasvegas/3200-las-vegas-blvd,36.128119,-115.170220


#### How many locations?

In [9]:
len(df)

380

#### Create a mapping of state abbreviations to full state names using the us library

In [10]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [11]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [12]:
df["brand"] = place_formal

---

## Geography

#### Make it a geodataframe

In [13]:
df_geo = df.copy()

In [14]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [15]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)